# ESCUELA POLITÉCNICA NACIONAL
# FACULTAD DE INGENIERÍA EN SISTEMAS

## Proyecto Bimestral: Sistema de Recuperación de Información basado en Reuters-21578

### Autor: Soto Condoy Bruce Andrés

## 1. INTRODUCIÓN
El objetivo de este proyecto es diseñar, construir, programar y desplegar un Sistema de Recuperación de Información (SRI) utilizando el corpus Reuters-21578. El proyecto se dividirá en varias fases, que se describen a continuación.



## 2. FASES DEL PROYECTO
Para realizar el proyecto se ocuparon libreías, las cuales seran de mucha ayuda al momento de realizar cada una de las fases del proyecto. Antes de continuar se muestran cada una de ellas y su respectiva función

#### Importación de librerías


In [3]:
import os  #modulo para interactuar con el sistema operativo
import numpy as np  #biblioteca para arrays y funciones matematicas
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer  #funciones para conteo de palabras y usar TF-IDF
import pandas as pd  #biblioteca para analisis y manipulacion de datos
from collections import defaultdict  #clase para diccionarios
import json  #modulo para trabajar con datos en formato JSON
import re  #modulo para manipulacion de texto
import nltk  #biblioteca para procesamiento del lenguaje natural
from nltk.tokenize import word_tokenize  # funcioun para dividir texto en palabras
from nltk.stem import PorterStemmer  #clase para reducir palabras a su raiz
from sklearn.metrics.pairwise import cosine_similarity  #funcion para calcular similitud coseno
from sklearn.metrics import precision_score, recall_score, f1_score  #funciones para evaluar modelos de clasificacion

###  2.1. Adquisición de Datos
Se descargó el archivo reuter.rar contenido en el url: https://github.com/ivan-carrera/ir24a/blob/main/proj01/data/reuters.rar <br>
Posterior a eso se descomprimieron todos los archivos dentro del directorio para seguir con el proyecto.

###  2.2. Preprocesamiento

#### Obtención de las stopwords
Para obetener las stopwords se uso el archivo *stopwords* que se encuentra en el directorio y se las guardo en el array "stop_words"

In [4]:
with open('data/reuters/stopwords', 'r') as file:
    content = file.read()
    stop_words = content.split('\n')

stop_words = [word.strip() for word in stop_words if word.strip()]
print(stop_words[:])

['a', "a's", 'able', 'about', 'above', 'according', 'accordingly', 'across', 'actually', 'after', 'afterwards', 'again', 'against', "ain't", 'all', 'allow', 'allows', 'almost', 'alone', 'along', 'already', 'also', 'although', 'always', 'am', 'among', 'amongst', 'an', 'and', 'another', 'any', 'anybody', 'anyhow', 'anyone', 'anything', 'anyway', 'anyways', 'anywhere', 'apart', 'appear', 'appreciate', 'appropriate', 'are', "aren't", 'around', 'as', 'aside', 'ask', 'asking', 'associated', 'at', 'available', 'away', 'awfully', 'b', 'be', 'became', 'because', 'become', 'becomes', 'becoming', 'been', 'before', 'beforehand', 'behind', 'being', 'believe', 'below', 'beside', 'besides', 'best', 'better', 'between', 'beyond', 'both', 'brief', 'but', 'by', 'c', "c'mon", "c's", 'came', 'can', "can't", 'cannot', 'cant', 'cause', 'causes', 'certain', 'certainly', 'changes', 'clearly', 'co', 'com', 'come', 'comes', 'concerning', 'consequently', 'consider', 'considering', 'contain', 'containing', 'conta

#### Preprocesamiento de documentos
Dentro de la funcion "preprocess_document" se realiza lo siguiente:
 -  Se eliminan caracteres no deseados y se normaliza el texto (se convierte todo a minúsculas).
 - Se divide el texto en palabras utilizando word_tokenize de NLTK.
 - Se eliminan las stop words y se aplica stemming utilizando PorterStemmer de NLTK.

In [5]:
ps = PorterStemmer() #definir el stemmer fuera ya que se usara en el resto del codigo
def preprocess_document(content):
    text = re.sub(r'\s+', ' ', content)  #remover espacios extra
    text = re.sub(r'[^a-zA-Z]', ' ', text)  #mantener solo caracteres alfabeticos
    text = text.lower()  #convertir a minusculas
    
    tokens = word_tokenize(text, language='english') #tokenizacion
    
    tokens = [ps.stem(word) for word in tokens if word not in stop_words and len(word) > 1] #eliminar stop words y aplicar stemming

    return tokens


#### Preprocesamiento de archivos
La función "preprocess_files" recorre todos los archivos en el directorio de entrada, los procesa utilizando "preprocess_document", y guarda los resultados preprocesados en la carpeta de salida *data/reuters/preprocessed_data*.

In [6]:
def preprocess_files(input_folder, output_folder):
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)
    for filename in os.listdir(input_folder):
        if '.' not in filename:
            with open(os.path.join(input_folder, filename), 'r', encoding='latin1') as file:
                content = file.read()
                tokens = preprocess_document(content)
                output_filename = os.path.join(output_folder, f"{filename}")
                with open(output_filename, 'w') as output_file:
                    output_file.write(' '.join(tokens))
                    
preprocess_files('data/reuters/training', 'data/reuters/preprocessed_data') #llamada a la funcion indicando el path del archivo donde se encuentran los datos y el path en donde queremos que se guarden los archivos procesados

#### Lectura de datos preprocesados
La función "read_preprocessed_files" lee los documentos preprocesados desde la carpeta especificada y devuelve una lista de documentos y sus nombres de archivo.

In [7]:
def read_preprocessed_files(folder):
    documents = []
    filenames = []
    
    for filename in os.listdir(folder):
        if '.' not in filename:
            with open(os.path.join(folder, filename), 'r', encoding='latin1') as file:
                content = file.read()
                documents.append(content)
                filenames.append(filename)
    return documents, filenames

documents, filenames = read_preprocessed_files('data/reuters/preprocessed_data')


### 2.3. Representación de Datos en Espacio Vectorial

#### Aplicar Bag of Words (BoW)
La función "apply_bow" utiliza CountVectorizer de scikit-learn para vectorizar los documentos usando la técnica BoW.<br>
<br>
#### Aplicar TF-IDF
La función "apply_tfidf" utiliza TfidfVectorizer de scikit-learn para vectorizar los documentos usando la técnica TF-IDF.<br>

In [8]:
#aplicar BoW
def apply_bow(documents):
    vectorizer = CountVectorizer()
    X = vectorizer.fit_transform(documents)
    return X, vectorizer

#aplicar TF-IDF
def apply_tfidf(documents):
    vectorizer = TfidfVectorizer()
    X = vectorizer.fit_transform(documents)
    return X, vectorizer

X_bow, bow_vectorizer = apply_bow(documents)
X_tfidf, tfidf_vectorizer = apply_tfidf(documents)


### 2.4. Indexación

#### Construcción del índice invertido
En la función "build_inverted_index" se usa *defaultdict* de la biblioteca collections para almacenar el índice invertido. Para cada término en cada documento, si el valor del término es mayor que 0, agregamos el identificador del documento a la lista correspondiente en el diccionario.

In [9]:
def build_inverted_index(X, vectorizer, filenames):
    inverted_index = defaultdict(list)
    terms = vectorizer.get_feature_names_out()
    for doc_id, doc in enumerate(X.toarray()):
        for term_id, term_freq in enumerate(doc):
            if term_freq > 0:
                term = terms[term_id]
                inverted_index[term].append(filenames[doc_id])
    return inverted_index



#### Guardado del índice invertido
El índice invertido se guarda en la función "save_inverted_index" en un archivo JSON para facilitar su uso en futuras búsquedas.

In [10]:
def save_inverted_index(inverted_index, output_file):
    with open(output_file, 'w') as f:
        json.dump(inverted_index, f, indent=4) #guardar indice invertido en un archivo JSON

Llamado de las funciones para construir y guardar el índice invertido

In [11]:
#construir y guardar indice invertido para BoW
inverted_index_bow = build_inverted_index(X_bow, bow_vectorizer, filenames)
save_inverted_index(inverted_index_bow, 'data/reuters/inverted_index_bow.json')

# construir y guardar indice invertido para TF-IDF
inverted_index_tfidf = build_inverted_index(X_tfidf, tfidf_vectorizer, filenames)
save_inverted_index(inverted_index_tfidf, 'data/reuters/inverted_index_tfidf.json')

### 2.5. Diseño del Motor de Búsqueda

#### Preprocesamiento de la Consulta
La función "preprocess_query" limpia el texto de la consulta eliminando caracteres no alfabéticos y espacios extra, convirtiendo todo a minúsculas, y aplicando tokenización, eliminación de stop words y stemming. Esto ayuda a normalizar la consulta para mejorar la precisión en la búsqueda.

In [12]:
def preprocess_query(text):
    text = re.sub(r'\s+', ' ', text)  #remover espacios extra
    text = re.sub(r'[^a-zA-Z]', ' ', text)  #mantener solo caracteres alfabeticos
    text = text.lower()  #convertir a minusculas

    tokens_query = word_tokenize(text, language='english') #tokenizacion

    tokens_query = [ps.stem(word) for word in tokens_query if word not in stop_words and len(word) > 1] #eliminar stop words y aplicar stemming
    
    return ' '.join(tokens_query)


#### Búsqueda de Documentos Relevantes
La función "search" toma una consulta y busca documentos relevantes utilizando un índice invertido y un vectorizador. Preprocesa la consulta, encuentra documentos relevantes, calcula similitudes de coseno, y devuelve los documentos más relevantes ordenados por similitud, limitándose a aquellos con una similitud mayor a 0.35.

In [30]:
def search(query, inverted_index, vectorizer, X, filenames, top_n=None):
    query = preprocess_query(query)
    query_terms = query.split()
    
    filename_to_index = {filename: idx for idx, filename in enumerate(filenames)} #mapear nombres de archivos a sus indices
    relevant_docs = set()
    for term in query_terms:
        if term in inverted_index:
            relevant_docs.update(inverted_index[term])
    relevant_docs = [filename_to_index[doc] for doc in relevant_docs if doc in filename_to_index] #convertir nombres de archivos relevantes a indices
    if not relevant_docs:
        return [], []
    X_relevant = X[relevant_docs]
    filenames_relevant = [filenames[i] for i in relevant_docs]

    #procesar la consulta y calcular similitudes
    query_vector = vectorizer.transform([query])
    similarities = cosine_similarity(query_vector, X_relevant).flatten()
    
    #ordenar documentos por similitud y limitar a los que tengan una siminitud de mas del 35%
    ranked_indices = np.argsort(similarities)[::-1][:]
    ranked_filenames = [filenames_relevant[i] for i in ranked_indices if similarities[i] > 0.35]
    
    return ranked_filenames, similarities[ranked_indices]

#ejemplo
query = "soybean oilseed"

ranked_filenames_bow, similarities_bow = search(query,inverted_index_bow, bow_vectorizer, X_bow, filenames)
ranked_filenames_tfidf, similarities_tfidf = search(query, inverted_index_tfidf, tfidf_vectorizer, X_tfidf, filenames)

#mostrar resultados
print("Resultados para BoW:")
for filename, similarity in zip(ranked_filenames_bow, similarities_bow):
    category = ''
    if filename in processed_categories:
        category = processed_categories[filename]
    print(f"{filename} {similarity}{category}")
print("\nResultados para TF-IDF:")
for filename, similarity in zip(ranked_filenames_tfidf, similarities_tfidf):
    category = ''
    if filename in processed_categories:
        category = processed_categories[filename]
    print(f"{filename} {similarity}{category}")

Resultados para BoW:
5702 0.46694217336895677['oilse', 'soybean']
9617 0.44189395817353083['soybean', 'oilse']
7356 0.3926035266493783['soybean', 'oilse']
6906 0.3598560863424403['soybean', 'oilse']

Resultados para TF-IDF:
3540 0.46133600066185043['coconut oil', 'palm oil', 'veg oil', 'soybean', 'oilse']
5702 0.4584567560163928['oilse', 'soybean']
9617 0.42097743338966576['soybean', 'oilse']
7356 0.36308960159507786['soybean', 'oilse']
3888 0.36126966080973694['oilse']


### 2.6. Evaluación del Sistema
Para la evalucion del sistema se debe usar el archivo *cats.txt* el cual contiene cada una de las categorías a las que pertenece cada archivo, con esto se puede usar las metricas de evaluación como: recall, precisión y F1-score

#### Categorías
La función "load_categories" carga las categorías de los documentos en un diccionario, mapeando cada nombre de archivo a una lista de categorías.

In [14]:
def load_categories(file):
    categories = {}
    with open(file, 'r') as f:
        for line in f:
            parts = line.strip().split()
            if len(parts) > 1:
                categories[parts[0]] = parts[1:]
    return categories
categories = load_categories('data/reuters/cats.txt')
categories = {key: value for key, value in categories.items() if 'training' in key} #obtener solo de los documentos de training

#### Preprocesado de categorías
La función "preprocess_categories" limpia las categorías de los documentos eliminando caracteres no alfabéticos y espacios extra, convirtiendo todo a minúsculas, y aplicando tokenización, eliminación de stop words y stemming.

In [15]:
def preprocess_categories(categories):
    processed_categories = {}
    for filename, cats in categories.items():
        processed_cats = []
        for cat in cats:
            cat = re.sub(r'\s+', ' ', cat) #remover espacios extra
            cat = re.sub(r'[^a-zA-Z]', ' ', cat) #mantener solo caracteres alfabeticos
            cat = cat.lower() #convertir a minusculas
            
            tokens = word_tokenize(cat, language='english') #tokenizacion
            
            tokens = [ps.stem(word) for word in tokens if word not in stop_words and len(word) > 1] #eliminar stop words y aplicar stemming
            
            processed_cat = ' '.join(tokens)
            
            if processed_cat:
                processed_cats.append(processed_cat)
        
        base_filename = filename.split('/')[-1] #pasar de 'training/1' a '1'
        processed_categories[base_filename] = processed_cats
        
    return processed_categories

processed_categories = preprocess_categories(categories)

#### Evaluación del sistema
La función "evaluate_search_system" toma un conjunto de consultas y un vectorizador y calcula las métricas de precisión, recall y F1-score para el sistema de búsqueda. Para cada consulta, procesa la consulta, busca documentos relevantes, obtiene las categorías reales de los documentos relevantes, y calcula las métricas de precisión, recall y F1-score.

In [31]:
def evaluate_search_system(query_set, vectorizer, X, filenames, inverted_index):
    precision_scores = []
    recall_scores = []
    f1_scores = []

    for query in query_set:
        query_processed = preprocess_query(query)
        
        ranked_filenames, _ = search(query, inverted_index, vectorizer, X, filenames) #obtener documentos relevantes
        
        #obtener las categorias reales de los documentos relevantes
        actual_categories = []
        for filename in ranked_filenames:
            if filename in processed_categories:
                actual_categories.extend(processed_categories[filename])

        predicted_categories = [query_processed] * len(actual_categories) #todas las categorias predichas son las mismas de la query, ya que al devolverme los documentos relevantes lo toma de esa forma
        
        #calculo de metricas
        if len(actual_categories) == 0 and len(predicted_categories) == 0:
            precision = 1.0
            recall = 1.0
            f1 = 1.0
        else:
            precision = precision_score(actual_categories, predicted_categories, average='micro', zero_division=1)
            recall = recall_score(actual_categories, predicted_categories, average='micro', zero_division=1)
            f1 = f1_score(actual_categories, predicted_categories, average='micro', zero_division=1)

        precision_scores.append(precision)
        recall_scores.append(recall)
        f1_scores.append(f1)

    #promedios de las metricas
    avg_precision = np.mean(precision_scores)
    avg_recall = np.mean(recall_scores)
    avg_f1 = np.mean(f1_scores)

    return avg_precision, avg_recall, avg_f1

#evaluar con todas las categorias
all_categories = set()
for cats in categories.values():
    all_categories.update(cats)
query_set = list(all_categories)

precision_bow, recall_bow, f1_bow = evaluate_search_system(query_set, bow_vectorizer, X_bow, filenames, inverted_index_bow)
precision_tfidf, recall_tfidf, f1_tfidf = evaluate_search_system(query_set, tfidf_vectorizer, X_tfidf, filenames, inverted_index_tfidf)

print("Resultados de evaluacion usando BoW:")
print(f"Precision: {precision_bow:.4f}")
print(f"Recall: {recall_bow:.4f}")
print(f"F1-score: {f1_bow:.4f}")

print("\nResultados de evaluacion usando TF-IDF:")
print(f"Precision: {precision_tfidf:.4f}")
print(f"Recall: {recall_tfidf:.4f}")
print(f"F1-score: {f1_tfidf:.4f}")

Resultados de evaluacion usando BoW:
Precision: 0.6482
Recall: 0.6482
F1-score: 0.6482

Resultados de evaluacion usando TF-IDF:
Precision: 0.6538
Recall: 0.6538
F1-score: 0.6538


## Servidor con flask

In [ ]:
from flask import Flask, request, jsonify
from flask_cors import CORS

app = Flask(__name__) #guardar servidor en app
CORS(app)  #permitir solicitudes CORS de cualquier origen

api: str = '/api/v1/' #nombre de la api

#flask routing
@app.route(api+'', methods=['GET'])
def hello():
    response = {'hello': "Hello world.."}
    return jsonify(response)

@app.route(api+'search', methods=['POST'])
def buscar():
    data = request.get_json()
    query = data.get('query')
    results = {'results': f'Resultados para la búsqueda: {query}'}
    return jsonify(results)

#servidor principal
if __name__ == '__main__':
    app.run() #si se realiza camabios se actualiza automaticamente